In [1]:
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# os.environ["TF_XLA_FLAGS"] ="--tf_xla_auto_jit=2"
sys.path.append("/home/Developer/NCSN-TF2.0/")

import PIL
import utils, configs
import argparse
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import matplotlib.pyplot as plt

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from helper import plot_confusion_matrix, metrics

from datasets.dataset_loader import  *
from tqdm import tqdm
from sklearn.metrics import classification_report, average_precision_score
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from PIL import Image
from IPython.display import display
from matplotlib.pyplot import imshow
from datetime import datetime

import seaborn as sns
import plotly as py
import plotly.graph_objs as go


# example of calculating the frechet inception distance in Keras
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy import linalg
from scipy.linalg import sqrtm

# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.inception_v3 import preprocess_input
# from keras.datasets.mnist import load_data

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.datasets.mnist import load_data

from skimage.transform import resize

sns.set(style="darkgrid")
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 16

seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

tf.__version__

'2.2.0'

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
# scale an array of images to a new size
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = resize(image, new_shape, 0)
        # store
        images_list.append(new_image)
    return asarray(images_list)

# calculate frechet inception distance
def calculate_fid(model, images1, images2):
    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).

    Stable version by Dougal J. Sutherland.

    Params:
    -- mu1 : Numpy array containing the activations of the pool_3 layer of the
             inception net ( like returned by the function 'get_predictions')
             for generated samples.
    -- mu2   : The sample mean over activations of the pool_3 layer, precalcualted
               on an representive data set.
    -- sigma1: The covariance matrix over activations of the pool_3 layer for
               generated samples.
    -- sigma2: The covariance matrix over activations of the pool_3 layer,
               precalcualted on an representive data set.

    Returns:
    --   : The Frechet Distance.
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, "Training and test mean vectors have different lengths"
    assert sigma1.shape == sigma2.shape, "Training and test covariances have different dimensions"

    diff = mu1 - mu2

    # product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = "fid calculation produces singular product; adding %s to diagonal of cov estimates" % eps
        warnings.warn(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError("Imaginary component {}".format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean

# calculate frechet inception distance
# expects a list of image batches
def calculate_fid_stable(model, images1, images2):
    # calculate activations
    act1 = np.concatenate([model.predict(batch) for batch in images1])
    act2 = np.concatenate([model.predict(batch) for batch in images2])

    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
   
    return calculate_frechet_distance(mu1, sigma1, mu2, sigma2)

In [5]:
# # @tf.function
def preprocess_images_tf(images, shape, batch_sz=256):    
    imgs = []
    
    # Scale to shape + Inception preprocessing
    for i in range(0, images.shape[0], batch_sz):
        _img = tf.image.resize(images[i:i+batch_sz], shape)
        _img = preprocess_input(_img)
        imgs.append(_img)
    return imgs

In [6]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

In [7]:
# define two fake collections of images
images1 = randint(0, 255, 100*32*32*3)
images1 = images1.reshape((100,32,32,3))
images2 = randint(0, 255, 100*32*32*3)
images2 = images2.reshape((100,32,32,3))
print('Prepared', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')

Prepared (100, 32, 32, 3) (100, 32, 32, 3)


In [8]:
%%time

# resize + pre-process images
images1 = preprocess_images_tf(images1, (299,299), 10)
images2 = preprocess_images_tf(images2, (299,299), 10)

# fid between images1 and images1
fid = calculate_fid_stable(model, images1, images1)
print('FID (same): %.3f' % fid)


# fid between images1 and images2
fid = calculate_fid_stable(model, images1, images2)
print('Stable FID (different): %.3f' % fid)

FID (same): -0.000
Stable FID (different): 19.305
CPU times: user 6min 36s, sys: 9min 37s, total: 16min 14s
Wall time: 36.1 s


In [9]:
%%time

from numpy.random import shuffle
from keras.datasets import cifar10

# load cifar10 images
with tf.device("cpu"):
    (images1, _), (images2, _) = cifar10.load_data()
    # shuffle(images1)
    # images1 = images1[:10000]
    print('Loaded', images1.shape, images2.shape)
    # convert integer to floating point values
    images1 = images1.astype('float32')
    images2 = images2.astype('float32')

    # resize images
    images1 = preprocess_images_tf(images1, (299,299), batch_sz=256)
    images2 = preprocess_images_tf(images2, (299,299), batch_sz=256)

Loaded (50000, 32, 32, 3) (10000, 32, 32, 3)
CPU times: user 2min 28s, sys: 9min, total: 11min 29s
Wall time: 1min 59s


### Calculate Activations

In [11]:
%%time
with tf.device("gpu"):
    act1 = np.concatenate([model.predict(batch) for batch in images1])

CPU times: user 2min 36s, sys: 8min 23s, total: 10min 59s
Wall time: 1min 49s


In [10]:
%%time
with tf.device("gpu"):
    act2 = np.concatenate([model.predict(batch) for batch in images2])

CPU times: user 38.3 s, sys: 1min 44s, total: 2min 23s
Wall time: 25.4 s


In [13]:
%%time
# calculate mean and covariance statistics
mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)

CPU times: user 38.2 s, sys: 23.6 s, total: 1min 1s
Wall time: 2.9 s


In [14]:
%%time
# calculate fid
fid = calculate_frechet_distance(mu1, sigma1, mu2, sigma2)
print('FID: %.3f' % fid)

FID: 3.235
CPU times: user 4min 34s, sys: 7min 37s, total: 12min 11s
Wall time: 19.1 s


In [15]:
%%time
fid = calculate_frechet_distance(mu1, sigma1, mu1, sigma1)
print('FID (same): %.3f' % fid)

FID (same): -0.000
CPU times: user 4min 13s, sys: 7min 11s, total: 11min 24s
Wall time: 17.9 s


In [12]:
path = "./statistics/new_fid_stats_cifar10_train.npz"
# np.savez(path, mu=mu1, sigma=sigma1)

with np.load(path) as f:
    m, s = f['mu'][:], f['sigma'][:]

# assert np.allclose(m, mu1)
# assert np.allclose(s, sigma1)

## Getting files for a checkpoint
> ### Use evaluation experiment to generate samples first

In [8]:
import utils 

def get_command_line_args(_args):
    parser = utils._build_parser()

    parser = parser.parse_args(_args)

    utils.check_args_validity(parser)

    print("=" * 20 + "\nParameters: \n")
    for key in parser.__dict__:
        print(key + ': ' + str(parser.__dict__[key]))
    print("=" * 20 + "\n")
    return parser

# Make sure these params match the model you want to evaluate
args = get_command_line_args(["--checkpoint_dir=longleaf_models/",
                              "--filters=128",
                              "--dataset=cifar10",
                              "--sigma_low=0.01",
                              "--sigma_high=1",
                              "--split=95,5"
                             ])
configs.config_values = args
dir_statistics = './statistics'
save_dir, complete_model_name = utils.get_savemodel_dir()
save_dir, complete_model_name

Parameters: 

experiment: train
dataset: cifar10
model: refinenet
filters: 128
num_L: 10
sigma_low: 0.01
sigma_high: 1.0
sigma_sequence: geometric
steps: 200000
learning_rate: 0.001
batch_size: 128
samples_dir: ./samples/
checkpoint_dir: longleaf_models/
checkpoint_freq: 5000
resume: True
resume_from: -1
init_samples: 
k: 10
eval_setting: sample
ocnn: False
y_cond: False
max_to_keep: 2
split: ['95', '5']



('longleaf_models/refinenet128_cifar10_L10_SH1e+00_SL1e-02/train_95_5/',
 'refinenet128_cifar10_L10_SH1e+00_SL1e-02/train_95_5')

In [13]:
%%time

import pathlib
from imageio import imread

multiple = 10000
i = 1
step_ckpt = 0
fids = []
for i in tqdm(range(1, 21), desc="Ckpt:"):
    step_ckpt = i * multiple
    save_directory = '{}/{}/step{}/samples/'.format(dir_statistics, complete_model_name, step_ckpt)
    
    path = pathlib.Path(save_directory)
    files = list(path.glob('*.jpg')) + list(path.glob('*.png'))
    x = np.array([imread(str(fn)).astype(np.float32) for fn in files])

    with tf.device("cpu"):
        generated_images = preprocess_images_tf(x, (299,299), batch_sz=256)

    with tf.device("gpu"):
        sample_acts = np.concatenate([model.predict(batch) for batch in generated_images])

    mus, sigmas = sample_acts.mean(axis=0), cov(sample_acts, rowvar=False)

    # calculate fid
    fid = calculate_frechet_distance(m, s, mus, sigmas)
    fids.append(fid)
    
    print('Ckpt-%d: %.3f' % (i, fid))

Ckpt::   5%|▌         | 1/20 [00:29<09:22, 29.61s/it]

Ckpt-1: 82.658


Ckpt::  10%|█         | 2/20 [00:47<07:52, 26.24s/it]

Ckpt-2: 136.799


Ckpt::  15%|█▌        | 3/20 [01:05<06:41, 23.62s/it]

...

Ckpt-18: 113.988


Ckpt::  95%|█████████▌| 19/20 [05:39<00:17, 17.72s/it]

Ckpt-19: 113.356


Ckpt:: 100%|██████████| 20/20 [05:57<00:00, 17.85s/it]

Ckpt-20: 115.152
CPU times: user 1h 6min 2s, sys: 1h 32min 45s, total: 2h 38min 47s
Wall time: 5min 57s


In [14]:
min_idx = np.argmin(fids)
min_idx+1, fids[min_idx]

(5, 70.21384937160167)